# Extract daily weather variables based on unique ID 


## Setup

### Connect to Drive

In [ ]:
from google.colab import drive # import drive from google colab
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
rootPath = "/content/drive/My Drive/California FireTrends (2012-2020)"
os.chdir(rootPath)

### Connect to GEE API

In [ ]:
# initialize and connect to GEE 
from google.colab import auth
auth.authenticate_user()
!earthengine authenticate
import ee 
ee.Initialize()

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

geemap package not installed. Installing ...


### Connect to Cloud Bucket

In [ ]:
# Connect to google cloud 
! gcloud auth login

### Load Libraries 

In [ ]:
%%time 

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib
import matplotlib.pyplot as plt 
import folium
import rtree

In [ ]:
def listFiles_ByExt(rootPath, ext):
  '''
  retrieve file path + names based on extension
  '''
  file_list = []
  root = rootPath
  for path, subdirs, files in os.walk(root):
      for names in files: 
          if names.endswith(ext):
              file_list.append(os.path.join(path,names))
  return file_list

def createFolder(rootPath, folderName): 
  '''
  Create new folder in root path 
  '''
  folderPath = os.path.join(rootPath, folderName) 
  if not os.path.exists(folderPath):
      os.makedirs(folderPath)
  return folderPath 


def listFiles_ALL(rootPath):
  '''
  retrieve file path + names 
  '''
  file_list = []
  root = rootPath
  for path, subdirs, files in os.walk(root):
    for names in files: 
      file_list.append(os.path.join(path,names))
  return file_list

## Prepare Fire Perimeters

### Combine all interpolated surfaces into one file

In [ ]:
rootPath = os.path.join(r'Products', 'By_Fire')
inter_files = listFiles_ByExt(rootPath,'.geojson')

In [ ]:
outpath = os.path.join(r'Products', 'ALL_2012_2020')

gdf = pd.concat([
    gpd.read_file(shp)
    for shp in inter_files
    ]).pipe(gpd.GeoDataFrame)

# gdf.to_file(os.path.join(outpath, f'ALL_2012_2020.shp'))
# gdf.to_file(os.path.join(outpath, f'ALL_2012_2020.geojson'), driver='GeoJSON')

### Clip to CA Extent

Currently the study area is a bit outside state lines 

In [ ]:
gdf.crs

In [ ]:
# check; set CRS 
state_path = '/content/drive/My Drive/COVID_FireTrends/Data/US_states/'
states = gpd.read_file(state_path)
CA_states = states[(states.STATE_NAME == 'California')]
CA_states = CA_states.to_crs('EPSG:3310')
CA_states_buff = CA_states
CA_states_buff['geometry'] = CA_states.geometry.buffer(5000)

In [ ]:
CA_states_buff.to_file('Data/CA_Extent/California_5kmbuff.shp')

In [ ]:
## clip to california extent 
Interpolated_CA = gpd.overlay(gdf, CA_states_buff, how='intersection')

In [ ]:
# Interpolated_CA.to_file(os.path.join(outpath, f'ALL_2012_2020.shp'))
# Interpolated_CA.to_file(os.path.join(outpath, f'ALL_2012_2020.geojson'), driver='GeoJSON')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


### Create unique ID column (numeric)

{FIREID}{YR}{JD}

In [ ]:
Interpolated_CA.sort_values(by=['Year', 'Fire'])
# Interpolated_CA['FireID'] = Interpolated_CA.groupby(['Fire', 'Year']).cumcount()
Interpolated_CA['FireID'] = 1
mask = Interpolated_CA.groupby(['Fire','Year'])['FireID'].transform(lambda x : len(x)>0)
Interpolated_CA.loc[mask,'FireID'] = Interpolated_CA.loc[mask,['Fire','Year']].astype(str).sum(1).factorize()[0] + 1

In [ ]:
Interpolated_CA['Unique_ID'] =  Interpolated_CA['YRJD'].astype('str').str[2:] + Interpolated_CA['FireID'].astype('str').str.zfill(3)
Interpolated_CA['Unique_ID'] = Interpolated_CA['Unique_ID'].astype('int64')

In [ ]:
Interpolated_CA['FireID'].nunique()
Interpolated_CA.groupby(['Fire', 'Year']).ngroups

500

In [ ]:
Interpolated_CA.to_file(os.path.join(outpath, f'ALL_2012_2020.shp'))
Interpolated_CA.to_file(os.path.join(outpath, f'ALL_2012_2020.geojson'), driver='GeoJSON')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


### Export to Bucket and to Asset

In [ ]:
! gsutil -m cp -R 'Data/CA_Extent' gs://daily_fire_surfaces

Copying file://Data/CA_Extent/California_5kmbuff.prj [Content-Type=application/octet-stream]...
Copying file://Data/CA_Extent/California_5kmbuff.cpg [Content-Type=application/octet-stream]...
Copying file://Data/CA_Extent/California_5kmbuff.shp [Content-Type=application/x-qgis]...
Copying file://Data/CA_Extent/California_5kmbuff.shx [Content-Type=application/x-qgis]...
Copying file://Data/CA_Extent/California_5kmbuff.dbf [Content-Type=application/octet-stream]...
- [5/5 files][ 20.2 KiB/ 20.2 KiB] 100% Done                                    
Operation completed over 5 objects/20.2 KiB.                                     


In [ ]:
! earthengine upload table --asset_id=users/escaduto/CA_FireTrends/CA_Extent gs://daily_fire_surfaces/CA_Extent/California_5kmbuff.shp

In [ ]:
! gsutil -m cp -R 'Products/ALL_2012_2020' gs://daily_fire_surfaces

In [ ]:
! earthengine upload table --asset_id=users/escaduto/CA_FireTrends/DerivedPerimeters_2012_2020 gs://daily_fire_surfaces/ALL_2012_2020/ALL_2012_2020.shp

In [ ]:
# import feature collection asset 
fire_perimeters = ee.FeatureCollection('users/escaduto/CA_FireTrends/DerivedPerimeters_2012_2020')
CA_Extent = ee.FeatureCollection('users/escaduto/CA_FireTrends/CA_Extent')

In [ ]:
outpath = os.path.join(r'Products', 'ALL_2012_2020')
Interpolated_CA = gpd.read_file(os.path.join(outpath, f'ALL_2012_2020.geojson'))

In [ ]:
Interpolated_CA['FireID'].nunique()

500

## Functions to GET Layers from GEE

#### Cloud Masks

In [ ]:
def maskL8sr(image):  
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3);
    cloudsBitMask = (1 << 5);
    # Get the pixel QA band.
    qa = image.select('pixel_qa');
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) and (qa.bitwiseAnd(cloudsBitMask).eq(0));
    return image.updateMask(mask);

Landsat 7 Mask

In [ ]:
# cloud m
def cloudMaskL457(image):
  cloudShadowBitMask = (1 << 3);
  cloudsBitMask = (1 << 5);
  cloudsconfidenceBitMask = (1 << 7);
  qa = image.select('pixel_qa');
  mask = qa.bitwiseAnd(cloudsconfidenceBitMask).eq(0) and (qa.bitwiseAnd(cloudsBitMask).eq(0)) or (qa.bitwiseAnd(cloudShadowBitMask).eq(0));
  return image.updateMask(mask);


### A. Vegetation Indices (30m) 

1) MONTHLY NDVI, NDWI, NDMI

2) NDMI DELTA

3) MONTHLY EVI

4) ANNUAL NDVI, NDWI, NDMI

5) ANNUAL EVI




Landsat 8 Mask

#### [1] MONTHLY NDVI, NDWI, NDMI

In [ ]:
from datetime import datetime, timedelta

def getStartEndDates(yr, mth):
  date_time_obj = datetime.strptime(str(yr) + "-" + str(mth), '%Y-%m') # create datetime object
  d = date_time_obj - timedelta(days=60) # subtract 2 months
  newyr = str(d.year)
  newmth = str(d.month)
  startDate = newyr + "-" + newmth + "-01"
  endDate = str(yr) + "-" + str(mth) + "-01"
  return startDate, endDate

In [ ]:
def getMonthlyLandsat(yr, mth, LandsatCollection, Cloudmask):
  startDate, endDate = getStartEndDates(yr, mth)

  LandsatImage = ee.Image(ee.ImageCollection(LandsatCollection)
                  .filterDate(startDate, endDate)    
                  .sort('CLOUD_COVER')
                  .filterBounds(CA_Extent)
                  .map(Cloudmask)
                  .median());

  LandsatClip = LandsatImage.clip(CA_Extent)
  return LandsatClip 


def computeMonthlyVegIndices(yr, mth, LandsatCollection, Cloudmask, EVILayerName, swir_band, nir_band, red_band):
  Landsat = getMonthlyLandsat(yr, mth, LandsatCollection, Cloudmask)
  
  red = Landsat.select(red_band)
  nir = Landsat.select(nir_band)
  swir = Landsat.select(swir_band)

  NDVI = nir.subtract(red).divide(nir.add(red)).rename('NDVI'); # nir-red 
  NDMI = nir.subtract(swir).divide(nir.add(swir)).rename('NDMI'); # nir-swir
  NDMI_Delta = computeNDMIDelta(NDMI, mth, swir_band, nir_band)
  EVI = getMonthlylEVI(yr, mth, EVILayerName)

  Landsat = Landsat.addBands(NDMI).addBands(EVI).addBands(NDVI).addBands(NDMI_Delta)
  Landsat = Landsat.select(['NDVI', 'NDMI', 'EVI', 'NDMIDelta'])
  return (Landsat)

#### [2] NDMI DELTA

In [ ]:
def computeNDMIDelta(NDMI, mth, swir_band, nir_band):
  baseLine = ee.Image(ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
                  .filter(ee.Filter.calendarRange(2012 - 5, 2012,'year'))
                  .filter(ee.Filter.calendarRange(int(mth),int(mth),'month'))
                  .map(cloudMaskL457)
                  .filterBounds(CA_Extent)
                  .median());                                                                                         
                                                                                            
  baseLineClip = baseLine.clip(CA_Extent)

  swir = baseLineClip.select(swir_band);
  nir = baseLineClip.select(nir_band);
  
  baseLineNDMI = nir.subtract(swir).divide(nir.add(swir)).rename('NDMI_Base');  
  NDMIDelta = NDMI.select('NDMI').subtract(baseLineNDMI.select('NDMI_Base')).rename('NDMIDelta')      
  return NDMIDelta

#### [3] MONTHLY EVI

In [ ]:
def getMonthlylEVI(yr, mth, EVILayerName):   #'LANDSAT/LC08/C01/T1_32DAY_EVI'
  startDate, endDate = getStartEndDates(yr, mth)
  EVI_monthly_collection = ee.Image(ee.ImageCollection(EVILayerName)
                          .filterDate(startDate, endDate)
                          .filterBounds(CA_Extent)
                          .select('EVI')
                          .median());
                                  
  EVI_Monthly = EVI_monthly_collection.clip(CA_Extent).rename('EVI');  
  return EVI_Monthly    

#### [4] ANNUAL NDVI, NDWI, NDMI

In [ ]:
def getAnnualLandsat(yr, LandsatCollection, Cloudmask):
  start_yr = int(yr) - 1 
  Landsat_annual_Image = ee.Image(ee.ImageCollection(LandsatCollection)
                  .sort('CLOUD_COVER')
                  .filter(ee.Filter.date(f'{start_yr}-10-01', f'{yr}-04-30'))
                  .filterBounds(CA_Extent)
                  .map(Cloudmask)
                  .max());

  Landsat_Annual_Clip = Landsat_annual_Image.clip(CA_Extent)
  return Landsat_Annual_Clip

In [ ]:
def computeAnnualVegIndices(yr, LandsatCollection, Cloudmask, EVILayerName, swir_band, nir_band, red_band):
  Landsat = getAnnualLandsat(yr, LandsatCollection, Cloudmask)

  red = Landsat.select(red_band)
  nir = Landsat.select(nir_band)
  swir = Landsat.select(swir_band)

  NDVI = nir.subtract(red).divide(nir.add(red)).rename('Annual_NDVI');
  NDMI = nir.subtract(swir).divide(nir.add(swir)).rename('Annual_NDMI');
  EVI = getAnnualEVI(yr, EVILayerName)

  Landsat = Landsat.addBands(NDVI).addBands(NDMI).addBands(EVI)
  Landsat = Landsat.select(['Annual_NDVI', 'Annual_NDMI', 'Annual_EVI'])
  return (Landsat)

#### [5] ANNUAL EVI

In [ ]:
def getAnnualEVI(yr, EVILayerName): #'LANDSAT/LC08/C01/T1_32DAY_EVI'
  start_yr = int(yr) - 1 
  EVI_annual_collection = ee.Image(ee.ImageCollection(EVILayerName)
                          .filter(ee.Filter.date(f'{start_yr}-10-01', f'{yr}-04-30'))
                          .filterBounds(CA_Extent)
                          .select('EVI')
                          .max());
                                  
  EVI_Annual = EVI_annual_collection.clip(CA_Extent).rename('Annual_EVI');   
  return EVI_Annual

### B. Daily Weather (resample to 30m) 

1) Gridmet 

2) Gridmet FFWI

3) Daymet 

4) Daymet VPD & RH


#### [1] GridMET

In [ ]:
def getGRIDMET(yr, jd):
  gridmet_layers = ['pr', 'rmax', 'rmin', 'sph', 'srad', 'th', 
                    'tmmn','tmmx', 'vs', 'erc', 'bi', 'eto',
                    'fm100', 'fm1000', 'etr', 'vpd']

  gridmet_collection = ee.Image(ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')
                            .filter(ee.Filter.calendarRange(int(yr), int(yr),'year'))
                            .filter(ee.Filter.calendarRange(int(jd),int(jd),'DAY_OF_YEAR'))
                            .select(gridmet_layers)
                            .mosaic())

  gridmet = gridmet_collection.clip(CA_Extent) 

  gridmet = getFFWI(gridmet)
  return gridmet

#### [2] FFWI

In [ ]:
def getFFWI(gridmet):
  FFWI = gridmet.expression(
          "(b('rmax') < 10) ? (1 - (2 *( (0.03229 + (0.281073 * b('rmax')) - (0.000578 * b('rmax') * b('tmmx'))) / (30 * 1.0))) + ((1.5 * ( (0.03229 + (0.281073 * b('rmax')) - (0.000578 * b('rmax') * b('tmmx'))) /(30*1.0))**2)) - (0.5 * ((0.03229 + (0.281073 * b('rmax')) - (0.000578 * b('rmax') * b('tmmx'))) / (30 * 1.0))**3)) * (((1+b('vs')**2)**0.5) / 0.3002)" +
          ": (b('rmax') > 10 && b('rmax') <= 50) ? (1 - (2 *( (2.22749 + (0.160107* b('rmax')) - (0.01478* b('tmmx'))) / (30 * 1.0))) + ((1.5 * ( (2.22749 + (0.160107* b('rmax')) - (0.01478* b('tmmx'))) /(30*1.0))**2)) - (0.5 * ((2.22749 + (0.160107* b('rmax')) - (0.01478* b('tmmx'))) / (30 * 1.0))**3)) * (((1+b('vs')**2)**0.5) / 0.3002)" +
          ": (b('rmax') > 50) ? (1 - (2 *((21.0606 + (0.005565* (b('rmax')**2)) - (0.00035 * b('rmax') * b('tmmx')) - (0.483199 * b('rmax'))) / (30 * 1.0))) + ((1.5 * ( (21.0606 + (0.005565* (b('rmax')**2)) - (0.00035 * b('rmax') * b('tmmx')) - (0.483199 * b('rmax'))) /(30*1.0))**2)) - (0.5 * ((21.0606 + (0.005565* (b('rmax')**2)) - (0.00035 * b('rmax') * b('tmmx')) - (0.483199 * b('rmax'))) / (30 * 1.0))**3)) * (((1+b('vs')**2)**0.5) / 0.3002)" +
          ": 0").rename('FFWI')

  gridmet_full = gridmet.addBands(FFWI)
  return gridmet_full

#### [3] DayMET

In [ ]:
def getDAYMET(yr, jd):
  daymet_layers = ['tmax', 'srad', 'vp', 'tmin', 'prcp']

  daymet_collection = ee.Image(ee.ImageCollection('NASA/ORNL/DAYMET_V3')
                            .filter(ee.Filter.calendarRange(int(yr), int(yr),'year'))
                            .filter(ee.Filter.calendarRange(int(jd),int(jd),'DAY_OF_YEAR'))
                            .select(daymet_layers)
                            .mosaic())

  daymet = daymet_collection.clip(CA_Extent)

  daymet = getVPD_RH(daymet)

  return daymet 

#### [4] VPD & RH

In [ ]:
def getVPD_RH(daymet):
  VP_amb = daymet.expression("610.78**((17.269 * tmin) / (237.3 + tmin))", {
    'tmin': daymet.select('tmin')
  })
    
  VP_sat = daymet.expression("610.78**((17.269 * (0.394 * tmin + 0.606 * tmax)) / (237.3 + (0.394 * tmin + 0.606 * tmax)))", {
    'tmin': daymet.select('tmin'),
    'tmax': daymet.select('tmax')
  })

  VP_Deficit = VP_sat.subtract(VP_amb).divide(1000).rename('DayMET_VPD')
  Rel_Humidity = VP_amb.divide(VP_sat).multiply(100).rename('DayMET_RH')

  daymet_full = daymet.addBands(VP_Deficit).addBands(Rel_Humidity);
  return daymet_full

### C. Topography (resample to 30m)

1) DEM, Slope, Aspect

2) Multiscale TPI 

3) Topo Diversity 

#### [1] DEM, Slope, Aspect

In [ ]:
DEM_Image = ee.Image('USGS/NED').select('elevation');

DEM = DEM_Image.clip(CA_Extent).rename('DEM')
Slope = ee.Terrain.slope(DEM).rename('Slope')
Aspect = ee.Terrain.aspect(DEM).rename('Aspect')

topo = DEM.addBands(Slope).addBands(Aspect);

#### [2] TPI 

In [ ]:
# 270 m (Valley (-) Ridge (+))
TPI_Image = ee.Image('CSP/ERGo/1_0/US/mTPI').select('elevation');

TPI = TPI_Image.clip(CA_Extent) 

# ridge > + 1 STDEV
# 2 upper slope > 0.5 STDV =< 1 STDV
# 3 middle slope> -0.5 STDV, < 0.5 STDV, slope > 5 deg
# 4 flats slope >= -0.5 STDV, =< 0.5 STDV , slope <= 5 deg
# 5 lower slopes >= -1.0 STDEV, < 0.5 STDV
# 6 valleys < -1.0 STDV

In [ ]:
def getStats(operation, ras_Image, band_name):
  out = ras_Image.reduceRegion(**{
      'reducer': operation, 
      'bestEffort': True,
      'geometry': CA_Extent.geometry()
      });
  return out.getNumber(f'{band_name}').getInfo()


TPI_ST = getStats(ee.Reducer.stdDev(), TPI, 'elevation')
TPI_mean = getStats(ee.Reducer.mean(), TPI, 'elevation')
TPI_max = getStats(ee.Reducer.max(), TPI, 'elevation')
TPI_min = getStats(ee.Reducer.min(), TPI, 'elevation')

In [ ]:
thresholds = ee.Image([TPI_min, TPI_mean - TPI_ST, 
                       TPI_mean - TPI_ST/2, TPI_mean + TPI_ST/2, TPI_mean + TPI_ST, TPI_max]);

TPI_Zones = TPI.expression(f'b(0) <= {TPI_mean - (2*(TPI_ST))} ? 1 : \
                             b(0) < {TPI_mean - (1/2*(TPI_ST))} ? 2 : \
                             b(0) < {TPI_mean + (1/2*(TPI_ST))} ? 3 : \
                             b(0) < {TPI_mean + (1*(TPI_ST))} ? 4 : \
                             b(0) < {TPI_max} ? 5 : 0').rename('TPI')

ridge = TPI_Zones.select(['TPI']).eq(1)
valley = TPI_Zones.select(['TPI']).eq(5)

####[3] Topo Diversity

In [ ]:
topo_diversity_Image = ee.Image('CSP/ERGo/1_0/Global/ALOS_topoDiversity').select('constant');

topo_diversity= topo_diversity_Image.clip(CA_Extent).rename('TOPO_DIV')
topo = topo.addBands(topo_diversity)

### D. LandFire (30m)

USE: ee.Reducer.countDistinct()

1) Vegetation Type 

2) Vegetation Height 

#### [1] Vegetation Type

In [ ]:
EVT_Image = ee.Image(ee.ImageCollection('LANDFIRE/Vegetation/EVT/v1_4_0').select('EVT').mosaic())

EVT = EVT_Image.clip(CA_Extent).rename('EVT')

In [ ]:
print(EVT.bandNames().getInfo())

['EVT']


#### [2] Vegetation Height

In [ ]:
EVH_Image = ee.Image(ee.ImageCollection('LANDFIRE/Vegetation/EVH/v1_4_0').select('EVH').mosaic())

EVH = EVH_Image.clip(CA_Extent).rename('EVH')

In [ ]:
Veg_TPI = TPI_Zones.addBands(EVH).addBands(EVT);

### E. Cost Distance to human settlement (resample to 30m)

In [ ]:
roads = ee.FeatureCollection('TIGER/2016/Roads');

# // Create a source image (palm oil mill=1, others=0)
sources = ee.Image().toByte().paint(roads, 1);

# // Mask the sources image with itself.
sources = sources.updateMask(sources);
sources = sources.clip(CA_Extent)

# // The cost data is generated from slope 
cover = Slope.select(0);

# // Variable cost
cost = cover.lte(10) and (cover.gt(0)).multiply(1).add(
       cover.lte(30) and (cover.gt(10)).multiply(2)).add(
       cover.lte(50) and (cover.gt(30)).multiply(3)).add(
       cover.lte(60) and (cover.gt(50)).multiply(4)).add(
       cover.lte(100) and (cover.gt(60)).multiply(5))

# // Compute the cumulative cost to traverse the land cover.
cumulativeCost = cost.cumulativeCost(**{
  'source': sources,
  'maxDistance': 10 * 1000 
}).rename('travelCost')


## For each instance of fire, GET layers, Mosaic, Reduce, Save

Make sure all layers are aligned, with correct CRS projection and resolution

In [ ]:
# import feature collection asset 
fire_perimeters = ee.FeatureCollection('users/escaduto/CA_FireTrends/DerivedPerimeters_2012_2020')
CA_Extent = ee.FeatureCollection('users/escaduto/CA_FireTrends/CA_Extent')

In [ ]:
outpath = os.path.join(r'Products', 'ALL_2012_2020')
Interpolated_CA = gpd.read_file(os.path.join(outpath, f'ALL_2012_2020.geojson'))

In [ ]:
Interpolated_CA.columns

Index(['JulianDay', 'YRJD', 'Fire', 'Year', 'Month', 'Day', 'Date',
       'Area (acres)', 'Area (ha)', 'Time_Min', 'Time_Max', 'Area (acre',
       'STATE_NAME', 'DRAWSEQ', 'STATE_FIPS', 'SUB_REGION', 'STATE_ABBR',
       'FireID', 'Unique_ID', 'geometry'],
      dtype='object')

In [ ]:
def exportGEE(features, var, yrjd):
  exporttask = ee.batch.Export.table.toDrive(**{
  'collection': features,
  'folder': "GoogleEE_VariableExtraction_10012020",
  'fileNamePrefix': f'{yrjd}_{var}',
  # 'selectors':(["Aspect","DEM","Slope"]),
  });

  exporttask.start()
  print(f'exporting {yrjd}_{var}.csv')


In [ ]:
def GEEReducer(IMG, collection, reducer, scale):
  reduced = IMG.reduceRegions(**{
    'collection': collection,
    'reducer': reducer,
    'scale': scale,
  })
  return reduced

In [ ]:
# 'LANDSAT/LE07/C01/T1_SR'
# 'LANDSAT/LC08/C01/T1_SR'

def zonalStatsByDay(yr_jd_list, LandsatCollection, Cloudmask, EVILayerName, swir_band, nir_band, red_band):
  for yrjd in yr_jd_list:
    yr_jd_dt = datetime.strptime(yrjd, '%Y%j') 
    year = yr_jd_dt.year
    dy = yr_jd_dt.day
    mnth = yr_jd_dt.month
    mnth = str(mnth).zfill(2)
    jd = yrjd[-3:]
    print(jd, mnth, year)

    day_perimeter = fire_perimeters.filter(ee.Filter.eq('YRJD', yrjd))
    
    # Weather 
    daymet = getDAYMET(year, jd)
    gridmet = getGRIDMET(year, jd)

    # Vegetation 
    monthly_indices = computeMonthlyVegIndices(year, mnth, LandsatCollection, Cloudmask, EVILayerName, swir_band, nir_band, red_band)
    annual_indices = computeAnnualVegIndices(year, LandsatCollection, Cloudmask, EVILayerName, swir_band, nir_band, red_band)

    allLayers = monthly_indices.addBands(annual_indices).addBands(gridmet).addBands(topo).addBands(cumulativeCost)
    all_Features = GEEReducer(allLayers, day_perimeter, ee.Reducer.mean(), 10)

    exportGEE(all_Features, 'dailyfire_zonalstats', yrjd)


#### Daymet 2012-2019

In [ ]:
def daymet_zonalStatsByDay(yr_jd_list):
  for yrjd in yr_jd_list:
    yr_jd_dt = datetime.strptime(yrjd, '%Y%j') 
    year = yr_jd_dt.year
    dy = yr_jd_dt.day
    mnth = yr_jd_dt.month
    mnth = str(mnth).zfill(2)
    jd = yrjd[-3:]
    print(jd, mnth, year)

    day_perimeter = fire_perimeters.filter(ee.Filter.eq('YRJD', yrjd))
    
    # Weather 
    daymet = getDAYMET(year, jd)
    daymet_Features = GEEReducer(daymet, day_perimeter, ee.Reducer.mean(), 10)

    exportGEE(daymet_Features, 'DAYMET', yrjd)

In [ ]:
daymet_fires = Interpolated_CA[Interpolated_CA['Year'] <= 2019]
yr_jd_list  = daymet_fires['YRJD'].unique().tolist()
yr_jd_list.sort(reverse=True)

In [ ]:
daymet_zonalStatsByDay(yr_jd_list[158:])

#### Count Distinct (TPI, EVH, EVT) [**ARCPY for now]

In [ ]:
#[TODO] Fuel Type, Height, TPI, Fuel Model 
# LandFire + TPI (Percent by unique value)
#Veg_TPI_Features = GEEReducer(TPI_Zones, day_perimeter, ee.Reducer.countDistinct(), 30)
#ee.Reducer.group(**{'reducer': ee.Reducer.count(),'groupField':0,'groupName':'TPI'})

# exportGEE(Veg_TPI_Features, 'Veg_TPI', yrjd)

#### Landsat 8 2013-2020

In [ ]:
L8_fires = Interpolated_CA[Interpolated_CA['Year'] >= 2013]
yr_jd_list  = L8_fires['YRJD'].unique().tolist()
yr_jd_list.sort(reverse=True)

In [ ]:
LandsatCollection = 'LANDSAT/LC08/C01/T1_SR'
Cloudmask = maskL8sr
EVILayerName='LANDSAT/LC08/C01/T1_32DAY_EVI'
swir_band = 'B6'
nir_band = 'B5'
red_band = 'B4'

zonalStatsByDay(yr_jd_list, LandsatCollection, Cloudmask, EVILayerName, swir_band, nir_band, red_band)

#### Landsat 7 2012-2013

In [ ]:
L7_fires = Interpolated_CA[Interpolated_CA['Year'] < 2013]
yr_jd_list = L7_fires['YRJD'].unique().tolist()
yr_jd_list.sort(reverse=True)

In [ ]:
LandsatCollection = 'LANDSAT/LE07/C01/T1_SR'
Cloudmask = cloudMaskL457
EVILayerName= 'LANDSAT/LE07/C01/T1_32DAY_EVI'
swir_band = 'B5'
nir_band = 'B4'
red_band = 'B3'

zonalStatsByDay(yr_jd_list, LandsatCollection, Cloudmask, EVILayerName, swir_band, nir_band, red_band)

In [ ]:
os.getcwd()

'/content/drive/My Drive/California_FireTrends'

## Visualize Layers

In [ ]:
visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
};

ndviParams ={
  'min': -1,
  'max': 1,
  'palette': ['ffff00', '330033','7fffd4']
};

Map = emap.Map(center=[38.50178453635526,-122.74843617724784], zoom=10)
# Map.addLayer(TPI_Zones, {'palette': ['ffff00', '330033','7fffd4', 
#                                                          'b99879','cc8e7f', '268b07']}, 'TPI_class')
Map.addLayer(landsat, visParams , 'landsat')
# Map.addLayer(valley, {'min':0, 'max':1, 'palette': ['ffff00', '330033']} , 'valley')
# Map.addLayer(ridge, {'min':0, 'max':1, 'palette': ['ffff00', '330033']} , 'ridge')
Map.addLayer(cumulativeCost, {'min': 0, 'max': 5e3, 'palette': ['34a853', 'fbc034', 'ea4335', '4285f4']} , 'cumulativeCost')
Map.addLayerControl()
Map

## Post-Processing: CSV

- By year 
- All 


### Merge daily files

In [ ]:
rootPath = "/content/drive/My Drive/GoogleEE_VariableExtraction_10012020"

In [ ]:
import glob, os

# gridmet, indices, topo, cost distance
vars_csv = glob.glob(os.path.join(rootPath, '*dailyfire_zonalstats.csv'))
combined_vars = pd.concat([pd.read_csv(f) for f in vars_csv])

In [ ]:
outpth_all = 'Variables/CSVs'
combined_vars.to_csv(os.path.join(outpth_all, "combinedvars_2012_2020.csv"))

In [ ]:
# daymet 
daymet_csv = glob.glob(os.path.join(rootPath, '*_DAYMET.csv'))
combined_daymet = pd.concat([pd.read_csv(f) for f in daymet_csv])
combined_daymet.to_csv(os.path.join(outpth_all, "daymet_2012_2019.csv"))

Percent Count do with arcpy

In [ ]:
import os
# TPI, EVH, EVT, fuel model (convert to decimal ratio)
outpath = os.path.join(r'Products', 'ALL_2012_2020')
Interpolated_CA = gpd.read_file(os.path.join(outpath, f'ALL_2012_2020.geojson'))

In [ ]:
Interpolated_CA_4326 = Interpolated_CA.to_crs('EPSG:4326')

In [ ]:
Interpolated_CA_4326['Group_ID'] = Interpolated_CA_4326['Unique_ID'].astype('str').str.zfill(8)

In [ ]:
Interpolated_CA_4326.to_file(os.path.join(outpath, f'ALL_2012_2020_WGS84.shp'))
Interpolated_CA_4326.to_file(os.path.join(outpath, f'ALL_2012_2020_WGS84.geojson'), driver='GeoJSON')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


### Cleanup & Compute Ratios (TPI, FBFM, EVT, EVH)

#### CBD

In [ ]:
CBD_df = pd.read_csv('Variables/ExtractVar_CSV/CBD/CBD_Extracted.csv', index_col=0).reset_index()
CBD_df.columns = ['OID_', 'Group_ID', 'ZONE_CODE', 'COUNT', 'AREA', 'CBD']
CBD_df = CBD_df[['Group_ID', 'CBD']]
CBD_df['Unique_ID'] = CBD_df['Group_ID'].astype('int64')
CBD_df = CBD_df.drop(columns=['Group_ID'])

In [ ]:
CBD_df.dtypes

CBD          float64
Unique_ID      int64
dtype: object

#### TPI

In [ ]:
TPI_df = pd.read_csv('Variables/ExtractVar_CSV/TPI/TPI_Extracted.csv', index_col=0).reset_index()
TPI_df.columns = ['OID_', 'Group_ID', 'Valley', 'LowerSlope', 'MidSlope', 'UpperSlope', 'Ridge']
TPI_df = TPI_df[['Group_ID', 'Valley', 'LowerSlope', 'MidSlope', 'UpperSlope', 'Ridge']]
TPI_df['Unique_ID'] = TPI_df['Group_ID'].astype('int64')
TPI_df = TPI_df.drop(columns=['Group_ID'])

In [ ]:
# Percent Ratio 
TPI_df = TPI_df.set_index('Unique_ID')
TPI_df = TPI_df.div(TPI_df.sum(axis=1), axis=0)
TPI_df = TPI_df.reset_index()

In [ ]:
len(TPI_df)

3996

#### FBFM

In [ ]:
FBFM_df = pd.read_csv('Variables/ExtractVar_CSV/FBFM/FBFM_Extracted.csv', index_col=0).reset_index()
FBFM_df = FBFM_df.drop(columns=['OID_', 'URBAN',	'SNOW_ICE',	'AGRICULTUR',	'WATER',	'BARREN']) 
FBFM_df['Unique_ID'] = FBFM_df['GROUP_ID'].astype('int64')
FBFM_df = FBFM_df.drop(columns=['GROUP_ID'])

In [ ]:
# Percent Ratio 
FBFM_df = FBFM_df.set_index('Unique_ID')
FBFM_df = FBFM_df.div(FBFM_df.sum(axis=1), axis=0)
FBFM_df = FBFM_df.reset_index()

In [ ]:
len(FBFM_df)

3996

#### EVH

In [ ]:
EVH_df = pd.read_csv('Variables/ExtractVar_CSV//EVH/EVH_Extracted.csv', index_col=0).reset_index()
EVH_df = EVH_df.drop(columns=['OID_']) 
EVH_df['Unique_ID'] = EVH_df['GROUP_ID'].astype('int64')
EVH_df = EVH_df.drop(columns=['GROUP_ID'])

In [ ]:
# Percent Ratio 
EVH_df['shortHeight'] = EVH_df['VALUE_101'] + EVH_df['VALUE_104']
EVH_df['medShortHeight'] = EVH_df['VALUE_102'] + EVH_df['VALUE_105']
EVH_df['medHeight'] = EVH_df['VALUE_103'] + EVH_df['VALUE_104'] + EVH_df['VALUE_107']
EVH_df['TreeHeight_5m'] = EVH_df['VALUE_108']
EVH_df['TreeHeight_10m'] = EVH_df['VALUE_109']
EVH_df['TreeHeight_25m'] = EVH_df['VALUE_110']
EVH_df['TreeHeight_50m'] = EVH_df['VALUE_111']
EVH_df['TreeHeight_50m+'] = EVH_df['VALUE_112']

EVH_df = EVH_df[['Unique_ID', 'shortHeight', 'medShortHeight', 'medHeight', 'TreeHeight_5m', 
        'TreeHeight_10m', 'TreeHeight_25m', 'TreeHeight_50m', 'TreeHeight_50m+']]

In [ ]:
# Percent Ratio 
EVH_df = EVH_df.set_index('Unique_ID')
EVH_df = EVH_df.div(EVH_df.sum(axis=1), axis=0)
EVH_df = EVH_df.reset_index()

In [ ]:
len(EVH_df)

3996

#### EVT

In [ ]:
EVT_df = pd.read_csv('Variables/ExtractVar_CSV/EVT/EVT_Extracted.csv', index_col=0).reset_index()
EVT_meta = pd.read_csv('Variables/ExtractVar_CSV/EVT/US_130EVT_02092015.csv', index_col=0).reset_index()
EVT_df['Unique_ID'] = EVT_df['GROUP_ID'].astype('int64')
EVT_df = EVT_df.drop(columns=['OID_', 'GROUP_ID'])

In [ ]:
EVT_meta = EVT_meta[['VALUE', 'EVT_PHYS', 'EVT_CLASS']]
EVT_meta['EVT_PHYS'] = np.where(EVT_meta['EVT_PHYS'] == 'Agricultural', EVT_meta['EVT_CLASS'], EVT_meta['EVT_PHYS'])
EVT_meta = EVT_meta[['VALUE', 'EVT_PHYS']]

In [ ]:
EVT_meta = EVT_meta.replace(['Conifer', 'Conifer-Hardwood', 'Open tree canopy'], 'Conifer')
EVT_meta = EVT_meta.replace(['Hardwood', 'Hardwood-Conifer'], 'Hardwood')
EVT_meta = EVT_meta.replace(['Barren', 'Non-vegetated'], 'Barren')
EVT_meta = EVT_meta.replace(['Barren', 'Non-vegetated'], 'Barren')
EVT_meta = EVT_meta.replace(['Grassland', 'Herbaceous - grassland', 'Exotic Herbaceous', 'Herbaceous / Nonvascular-dominated'], 'Grassland')
EVT_meta = EVT_meta.replace(['Shrubland', 'Exotic Tree-Shrub'], 'Shrubland')
EVT_meta = EVT_meta.replace(['Riparian'], 'Riparian')
EVT_meta = EVT_meta.replace(['Sparsely Vegetated'], 'Sparsely Vegetated')
EVT_meta = EVT_meta.replace(['Snow-Ice', 'Nodata', 'Open Water', 'Quarries-Strip Mines-Gravel Pits', 'Other'], 'Other')
EVT_meta = EVT_meta.replace(['Developed', 'Developed-High Intensity', 'Developed-Low Intensity', 'Developed-Medium Intensity', 'Developed-Roads'], 'Developed')

evt_value_info = EVT_meta.groupby(['EVT_PHYS'])['VALUE'].apply(list).reset_index()

In [ ]:
def prepend(lst, str): 
    # Using format() 
    str += '{0}'
    lst = [str.format(i) for i in lst] 
    return(lst) 

In [ ]:
column_list = EVT_df.columns.tolist()

def updateEVTColumns(column_list, evt_value_info, EVT_df):
  for index, row in evt_value_info.iterrows():
    ID_list = prepend(row.VALUE, 'VALUE_')
    ID_list = [x for x in ID_list if x in column_list]
    EVT_df[row.EVT_PHYS] = EVT_df[ID_list].sum(axis=1)
  return EVT_df

In [ ]:
list_col = evt_value_info['EVT_PHYS'].tolist()
list_col.append('Unique_ID')

In [ ]:
EVT_df = updateEVTColumns(column_list, evt_value_info, EVT_df)
EVT_df = EVT_df[list_col]

In [ ]:
# Percent Ratio 
EVT_df = EVT_df.set_index('Unique_ID')
EVT_df = EVT_df.div(EVT_df.sum(axis=1), axis=0)
EVT_df = EVT_df.reset_index()

In [ ]:
len(EVT_df)

3965

### Merge ALL

####2012-2020 (w/out Daymet)

In [ ]:
vars_df = pd.read_csv('Variables/CSVs/combinedvars_2012_2020.csv', index_col=0).reset_index()

In [ ]:
import pandas as pd
from functools import reduce

df_list = [vars_df, CBD_df, FBFM_df, TPI_df, EVH_df, EVT_df]

gridmet_2012_2020 = reduce(lambda  left,right: pd.merge(left,right,on=['Unique_ID'],
                                            how='left'), df_list)

In [ ]:
gridmet_2012_2020['FireID'].nunique()
# 408

500

In [ ]:
gridmet_2012_2020 = gridmet_2012_2020.fillna(0)
gridmet_2012_2020 = gridmet_2012_2020.drop(columns=['index', 'system:index', 'Area (ac_1', 'DRAWSEQ'])

In [ ]:
gridmet_2012_2020.to_csv('Variables/Final_Variables/gridmet_2012_2020.csv')

####2012-2019 (w/Daymet)

In [ ]:
daymet_df = pd.read_csv('Variables/CSVs/daymet_2012_2019.csv', index_col=0).reset_index()
#len 361

In [ ]:
daymet_df = daymet_df[['DayMET_RH', 'DayMET_VPD', 
            'Unique_ID', 'FireID', 'prcp', 'srad',
            'tmax', 'tmin', 'vp']]

In [ ]:
df_list = [daymet_df, CBD_df, FBFM_df, TPI_df, EVH_df, EVT_df]

daymet_2012_2019 = reduce(lambda  left,right: pd.merge(left,right,on=['Unique_ID'],
                                            how='inner'), df_list)
#len 359

In [ ]:
len(daymet_2012_2019['FireID'].unique())
# len 351

296

In [ ]:
daymet_2012_2019.to_csv('Variables/Final_Variables/daymet_2012_2019.csv')

#### w/RAWS

In [ ]:
raws_csv_list = listFiles_ByExt(f"Variables/WeatherStation_FireSpread/data/Final_CSV", '.csv')
combined_csv = pd.concat([pd.read_csv(f) for f in raws_csv_list ])

In [ ]:
combined_csv

In [ ]:
combined_csv = combined_csv[combined_csv['Date'].notna()]
combined_csv['Date'] = combined_csv['Date'].astype('int64')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
import math
def calculateVPD(Temp_val, RH_val):
  '''
  returns VPD based on temp and rel humidity values
  '''
  VP_sat = 0.61078 * math.exp((Temp_val * 17.2694) / (Temp_val + 238.3))
  VP_air = (VP_sat * RH_val) / 100 
  VPD = VP_sat - VP_air 
  return VPD

In [ ]:
def convertTemp_FtoC(Temp_val):
  '''
  converts F to C temperature
  '''
  C_TEMP = ((Temp_val - 32) * 5 ) / 9 
  return C_TEMP

In [ ]:
night_list = []
day_list = ['6 am', '7 am', '8 am', '9 am', '10 am','11 am', '12 pm','1 pm', '2 pm','3 pm', '4 pm', '5 pm', '6 pm',
            '7 pm', '8 pm', '9 pm', '10 pm','11 pm', '12 am','1 am', '2 am','3 am', '4 am', '5 am']

def getDailyRAWSByFire(rim_df):
  '''
  converts hourly RAWS to daily (night vs day). keeps only day.
  '''
  rim_df = rim_df.dropna(subset=['Date'])
  criteria = [rim_df['Hour'].isin(day_list), rim_df['Hour'].isin(night_list)]
  values = ['Day', 'Night']
  rim_df['Day_Night']  = np.select(criteria, values, 0)
  day_raws_df = rim_df.groupby(by=['Day_Night', 'Date', 'Fire']).agg({'Total_Solar_Rad': 'mean', 
                                                                    'Wind_Avg_mph': 'mean',
                                                                    'Wind_Dir_Deg': 'mean',
                                                                    'Wind_Max_mph': ['mean', 'max'],
                                                                    'Air_Temp_Avg': ['mean', 'max'],
                                                                    'Fuel_Temp_Avg': 'mean',
                                                                    'Fuel_Moist_Per': 'mean',
                                                                    'Rel_Hum_Per': 'mean',
                                                                    'Dew_Point_Deg': 'mean',
                                                                    'Wet_Bulb': 'mean',
                                                                    'Total_Precip': 'mean'}).reset_index()
  day_raws_df.columns = ['Day_Night', 'Date', 'Fire', 'Total_Solar_Rad', 'Wind_Avg_mph',
       'Wind_Dir_Deg', 'Wind_Max_Avg', 'Wind_Max_mph', 'Air_Temp_Avg', 'Air_Temp_Max',
       'Fuel_Temp_Avg', 'Fuel_Moist_Per', 'Rel_Hum_Per', 'Dew_Point_Deg',
       'Wet_Bulb', 'Total_Precip']

  day_raws_df['Air_Temp_Celcius'] = convertTemp_FtoC(day_raws_df['Air_Temp_Avg'])
  day_raws_df['Air_Temp_Max_Celcius'] = convertTemp_FtoC(day_raws_df['Air_Temp_Max'])
  day_raws_df['VPD'] = day_raws_df.apply(lambda x: calculateVPD(x['Air_Temp_Celcius'],x['Rel_Hum_Per']),axis=1)
  day_raws_df = day_raws_df[day_raws_df['Day_Night'] == 'Day']
  day_raws_df['Date'] = day_raws_df['Date'].astype('str')
  day_raws_df['Date'] = pd.to_datetime(day_raws_df['Date'], format= '%Y%m%d')
  day_raws_df['Year'] =  day_raws_df['Date'].apply(lambda x: int(x.strftime('%Y')))
  day_raws_df['Date'] =  day_raws_df['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
  return day_raws_df

In [ ]:
dailyRAWS_2012_2020 = getDailyRAWSByFire(combined_csv)

In [ ]:
len(dailyRAWS_2012_2020['Fire'].unique())
dailyRAWS_2012_2020.groupby(['Fire', 'Year']).ngroups

155

In [ ]:
df_list = [dailyRAWS_2012_2020, gridmet_2012_2020]

merged_RAWS_2012_2020 = reduce(lambda  left,right: pd.merge(left,right,on=['Fire', 'Year', 'Date'],
                                            how='inner'), df_list)

In [ ]:
merged_RAWS_2012_2020['FireID'].nunique()

151

In [ ]:
merged_RAWS_2012_2020.to_csv('Variables/Final_Variables/RAWS_fullday_2012_2020.csv')